###### Can I predict the existence of subfields with some cool unsupervised learning algorithm? 

For starters, let's just use regular n-grams. A more advanced version would be to look for noun phrases or J&K POS tags.

In [1]:
#Need to add parent directoy to sys.path to find 'metadataDB'
import sys
sys.path.append('../../')

%matplotlib inline
# import matplotlib.pyplot as plt
import time
import numpy as np
# import scipy as sp
import re
from collections import Counter
import itertools
import random

# Natural language processing toolkit
# To use this, run nltk.download() and download 'stopwords'
# from nltk.corpus import stopwords
# s=stopwords.words('english') + ['']

# Machine learning
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.cluster import KMeans
from sklearn.decomposition import SparsePCA
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
# from sklearn import metrics
from sklearn.externals import joblib

# SQL
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from metadataDB.declareDatabase import *
from sqlalchemy import or_, and_

engine = create_engine("sqlite:///../../arXiv_metadata.db", echo=False)
Base.metadata.bind = engine
DBsession = sessionmaker(bind=engine)
session = DBsession()

In [2]:
# What are the available categories?
categories = sorted([x.name for x in session.query(Category)])
print categories 

[u'acc-phys', u'adap-org', u'alg-geom', u'ao-sci', u'astro-ph', u'astro-ph.CO', u'astro-ph.EP', u'astro-ph.GA', u'astro-ph.HE', u'astro-ph.IM', u'astro-ph.SR', u'atom-ph', u'bayes-an', u'chao-dyn', u'chem-ph', u'comp-gas', u'cond-mat', u'cond-mat.dis-nn', u'cond-mat.mes-hall', u'cond-mat.mtrl-sci', u'cond-mat.other', u'cond-mat.quant-gas', u'cond-mat.soft', u'cond-mat.stat-mech', u'cond-mat.str-el', u'cond-mat.supr-con', u'cs.AI', u'cs.AR', u'cs.CC', u'cs.CE', u'cs.CG', u'cs.CL', u'cs.CR', u'cs.CV', u'cs.CY', u'cs.DB', u'cs.DC', u'cs.DL', u'cs.DM', u'cs.DS', u'cs.ET', u'cs.FL', u'cs.GL', u'cs.GR', u'cs.GT', u'cs.HC', u'cs.IR', u'cs.IT', u'cs.LG', u'cs.LO', u'cs.MA', u'cs.MM', u'cs.MS', u'cs.NA', u'cs.NE', u'cs.NI', u'cs.OH', u'cs.PF', u'cs.PL', u'cs.RO', u'cs.SC', u'cs.SD', u'cs.SE', u'cs.SI', u'cs.SY', u'dg-ga', u'funct-an', u'gr-qc', u'hep-ex', u'hep-lat', u'hep-ph', u'hep-th', u'math-ph', u'math.AC', u'math.AG', u'math.AP', u'math.AT', u'math.CA', u'math.CO', u'math.CT', u'math.CV',

In [3]:
abstract_all_tmp = {'category': [], 'abstract': []}
# category_list = sorted(['atom-ph', 'quant-ph', 'optics', 'nlin', 'str-el', 'stat'])
category_list = sorted(['atom-ph'])
category_len = len(category_list)

start = time.time()
for item in category_list:
    query = session.query(Article_Category)\
                        .join(Category)\
                        .join(Article)\
                        .filter(Category.name.like('%' + item + '%'))
#     query = session.query(Article_Category)\
#                         .join(Category)\
#                         .join(Article)\
#                         .filter(Category.name.like('%' + item + '%'))
    result = [' '.join(x.article.abstract.split()) for x in query]
    abstract_all_tmp['abstract'].extend(result)
    abstract_all_tmp['category'].extend([item]*len(result))
print time.time() - start
# for item in query:
#     abstract_all['category'].append(item.category.name)
#     abstract_all['abstract'].append(' '.join(item.article.abstract.split()))
# print time.time() - start
# abstract_all['atom-ph'] = [x.article.abstract for x in query.all()]
# session.close_all()

18.7947340012


In [4]:
# Breakdown of categories?
count = Counter(abstract_all_tmp['category'])
for key, val in count.iteritems():
    print '{:<15}{}'.format(key, val)
print '{:<15}{}'.format('Total', len(abstract_all_tmp['abstract']))

atom-ph        9156
Total          9156


In [5]:
abstract_all = abstract_all_tmp

In [6]:
# ##Oops! How many overlapping articles do we have? I forgot that arXiv categories aren't unique.
# # Let's remove all duplicates.
# # This is slow but I am tired.

# counter_duplicate = Counter(abstract_all_tmp['abstract'])

# abstract_all = {'category': [], 'abstract': []}
# for cat, abstract in itertools.izip(abstract_all_tmp['category'], abstract_all_tmp['abstract']):
#     if counter_duplicate[abstract] == 1:
#         abstract_all['category'].append(cat)
#         abstract_all['abstract'].append(abstract)
# print len(abstract_all['category'])
# print len(abstract_all['abstract'])

In [7]:
# # Breakdown of categories? That's a lot of repetition!!!
# count = Counter(abstract_all['category'])
# for key, val in count.iteritems():
#     print '{:<15}{}'.format(key, val)
# print '{:<15}{}'.format('Total', len(abstract_all['abstract']))

In [8]:
# Train on 90% of the data. Random_state ensures that we always get the same result.
x_train, x_test, y_train, y_test = train_test_split(abstract_all['abstract'],
                                                    abstract_all['category'],
                                                    random_state=42,
                                                    train_size=0.9)

counter_train = Counter(y_train)

Okay, I lied, I'm starting with supervised learning (as a comparison). We're looking at ~60-70% accuracy for these cateogories.

In [9]:
# #SVC(kernel='linear') is good
# clf_supervised = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))),
#                            ('tfidf', TfidfTransformer()),
# #                            ('clf', LinearSVC())])
#                            ('clf', LinearSVC(C=1,penalty='l1',dual=False,))])
# start = time.time()
# clf_supervised.fit(x_train, y_train)
# print time.time() - start

# start = time.time()
# predict = clf_supervised.predict(x_test)
# print time.time() - start
# #print text_abstract_clf.predict(train_abstract)

In [10]:
# print(classification_report(y_test, predict))
# print('Accuracy score: %0.2f' % accuracy_score(y_test, predict))

Find the most important words.

In [11]:
# # Most important chunks. See http://scikit-learn.org/stable/auto_examples/text/document_clustering.html
# most_important_words = clf_supervised.named_steps['clf'].coef_.argsort()[:, ::-1]

# terms =  clf_supervised.named_steps['vect'].get_feature_names()
# for i in range(len(category_list)):
#     print "Category %s:" % (category_list[i])
#     print ', '.join([terms[x] for x in most_important_words[i, :20]])
#     print ''

Now, try KMeans clustering. 
See: http://scikit-learn.org/stable/auto_examples/text/document_clustering.html

In [12]:
# 20 is good
# n_clusters = 20
n_clusters = 20
# Reduce n_init to 10 for testing purposes.
clf_unsupervised = Pipeline([('vect', CountVectorizer(ngram_range=(1,3), stop_words='english')),
                             ('tfidf', TfidfTransformer()),
                             ('clf', KMeans(n_clusters=n_clusters, n_init=20))])
start = time.time()
clf_unsupervised.fit(x_train)
print time.time() - start

start = time.time()
predict_train = clf_unsupervised.predict(x_train)
predict = clf_unsupervised.predict(x_test)
print time.time() - start


620.14134407
4.18002891541


Save the model and predictor

In [13]:
start = time.time()
joblib.dump(clf_unsupervised, 'cluster-atom-ph-2.pkl', compress=3) 
print time.time() - start

34.0559208393


In [14]:
# # Which clusters most closely align with which the original categories?
# # Find the strongest correlation, and assign that cluster to the category. Iterate.
# # matrix_train = [[sum((a==cat and b==x for a,b in zip(y_train, predict_train)))
# #                    for x in range(0,n_clusters)] 
# #                    for cat in category_list]


# counter_category = Counter(y_train)
# counter_cluster = Counter(predict_train)
# accuracy_train_initial = np.array(
# #     [[sum((a==cat and b==x for a,b in zip(y_train, predict_train)))*1./counter_cluster[x]
#     [[sum((a==cat and b==x for a,b in zip(y_train, predict_train)))*1./counter_category[cat]
#            for x in range(0,n_clusters)] 
#            for cat in category_list])
# clusterToCategory = dict()
# # categoryToCluster = dict()

# for cluster, item in enumerate(np.argmax(accuracy_train_initial, axis=0).tolist()):
#     clusterToCategory[cluster] = category_list[item]


# # category_list_remaining = list(category_list)
# # cluster_list_remaining = range(0, n_clusters)
# # for x in range(0, len(category_list)):
# #     accuracy_train = np.array(
# #                         [[sum((a==cat and b==x for a,b in zip(y_train, predict_train)))*1./counter_cluster[x]
# #                            for x in cluster_list_remaining] 
# #                            for cat in category_list_remaining])

    
# #     # Find largest value in the category axis
# #     cat_ind, cluster_ind = np.unravel_index(np.argmax(accuracy_train), accuracy_train.shape)
# #     clusterToCategory[cluster_list_remaining[cluster_ind]] = category_list_remaining[cat_ind]
# #     categoryToCluster[category_list_remaining[cat_ind]] = cluster_list_remaining[cluster_ind]
    
# #     # Remove those entries from the lists.
# #     category_list_remaining.remove(category_list_remaining[cat_ind])
# #     cluster_list_remaining.remove(cluster_list_remaining[cluster_ind])
# # #     break

# # # The remaining clusters predict empty strings
# # for item in cluster_list_remaining:
# #     clusterToCategory[item] = ''
# # clusterToCategory_list = sorted(clusterToCategory.values())

In [15]:
# # The table is normalized by number of elements in each cluster.
# print 'Training data'
# print ('{:<10}' + '{:<10}'  *n_clusters).format('', *range(0, n_clusters))
# print ('{:<10}' + '{:<10}'  *n_clusters).format('', *[clusterToCategory[x] for x in range(0, n_clusters)])
# for cat, item in zip(category_list, accuracy_train_initial):
#     print ('{:<10}' + '{:<10.2}'*n_clusters).format(cat, *item)

In [16]:
# # Is there overlap between the clusters and existing categories ('ground truth')?
# matrix = [[sum((a==cat and b==x for a,b in zip(y_test, predict)))
#            for x in range(0,n_clusters)] 
#            for cat in category_list]

# print 'Test data'
# print ('{:<10}' + '{:<10}'  *n_clusters).format('', *range(0, n_clusters))
# print ('{:<10}' + '{:<10}'  *n_clusters).format('', *[clusterToCategory[x] for x in range(0, n_clusters)])
# for cat, item in zip(category_list, matrix):
#     print ('{:<10}' + '{:<10}'*n_clusters).format(cat, *item)
    
    
# # # Oops, this is the same as the confusion matrix?
# # tmp_reverse_category = dict([(y,x) for x,y in enumerate(category_list)])
# # y_test_num = [tmp_reverse_category[x] for x in y_test]
# # print ''
# # print 'Confusion matrix:'
# # print confusion_matrix(y_test_num, predict)

In [17]:
# # We can now make a prediction based on these categories.
# predict_category = [clusterToCategory[y] for y in predict]

# print(classification_report(y_test, predict_category))
# print('Accuracy score: %0.2f' % accuracy_score(y_test, predict_category))
# print confusion_matrix(y_test, predict_category)

In [18]:
# Most important chunks. See http://scikit-learn.org/stable/auto_examples/text/document_clustering.html
order_centroids = clf_unsupervised.named_steps['clf'].cluster_centers_.argsort()[:, ::-1]

terms =  clf_unsupervised.named_steps['vect'].get_feature_names()
for i in range(n_clusters):
    print "Cluster %d:" % (i)
    print ', '.join([terms[x] for x in order_centroids[i, :15]])
    print ''

Cluster 0:
recombination, rate, ion, dr, 2p, 2s, electron, storage, ions, storage ring, ring, rate coefficients, recombination rate, coefficients, fe, heavy, data, rate coefficient, heavy ion, 2s 2p

Cluster 1:
scattering, cross, section, cross section, resonances, scattering length, length, body, energy, wave, range, resonance, efimov, atoms, elastic, atom, universal, ultracold, feshbach, electron

Cluster 2:
rydberg, atoms, dipole, states, interactions, rydberg atoms, interaction, van der, der, waals, van der waals, der waals, van, atom, excitation, rydberg states, state, blockade, dipole dipole, range

Cluster 3:
atom, atomic, electron, atoms, laser, time, light, pulse, energy, model, experimental, photon, wave, results, using, electrons, phase, al, optical, pulses

Cluster 4:
ion, ions, trap, levels, transitions, radiative, state, energy, decay, lines, strengths, transition, electron, atomic, rates, calculations, results, ion trap, charged, emission

Cluster 5:
bose, condensate, ei